In [1]:
import src.view.pnl_view as pnlview
from multiprocessing import Pool, Process, Manager, cpu_count
from src.core.logger import Logger
from src.core.domain.security_loader import load_security
import datetime
from src.core.stopwatch import Stopwatch
from src.core.stopwatch_logger import StopwatchLogger
from src.synthetic_book_merger import SyntheticBookMerger
from src.synthetic_security_loader import SyntheticSecurityLoader
import src.sniper_opp_loader
from src.sniper_opp_loader import SniperOppLoader
from src.core.domain.book import get_book_level_changes
from src.core.domain.book import to_price_changes_only

from src.view import data_view as dview
from src.view.opp_view import OppView, to_opp_view, display_opp_view
from src.view.cell_view import CellView
from src.view.color import Color, Colors, WebColors
from src.pcap_location_params import PCapLocationParams
from src.core.postgres_connection import PostgresConnection
from src.view.cell_view import CellView
from src.market_date import *

import math
import pandas as pd
import numpy as np
import sqlite3
import time
import matplotlib.pyplot as plt
import os.path

# pandarallel.initialize(progress_bar=True, use_memory_fs=True)
pd.options.display.width = 920
pd.options.display.max_columns = 500
# pd.options.display.max_rows = 220
# pd.options.display.max_rows = 263
pd.options.display.max_rows = 320

,sid,symbol,expiration,sec_type,cme_sec_type,sec_group,asset,implied_matching,match_algo,display_factor,cash,tick_size,min_price,n_legs,segment_id
0,2602,HRCZ1-BUSZ1,2021-12-10 18:30:00,SPREAD,FUT,HR,HRC,True,F,0.010,20.0,100.0,0.0,2,76
1,53302,HRCZ1-HRCN4,2021-12-28 18:30:00,SPREAD,FUT,HR,HRC,True,F,0.010,20.0,100.0,0.0,2,76
2,99085,HDGZ1-HDGV3,2021-12-28 18:30:00,SPREAD,FUT,HD,HDG,False,F,0.010,20.0,100.0,0.0,2,76
3,63730,ALIN2-ALIZ3,2022-07-27 16:00:00,SPREAD,FUT,AL,ALI,True,K,0.010,25.0,25.0,0.0,2,76
4,2092,ALIF6,2026-01-28 17:00:00,FUTURE,FUT,AL,ALI,True,K,0.010,25.0,25.0,25.0,0,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8212,89463,EDPN2,2022-07-29 16:00:00,FUTURE,FUT,ST,EDP,True,F,0.010,0.0,1.0,1.0,0,76
8213,17552,HRCU1-HRCG2,2021-09-28 17:30:00,SPREAD,FUT,HR,HRC,True,F,0.010,20.0,100.0,0.0,2,76
8214,79190,SIN1-SIZ5,2021-07-28 17:25:00,SPREAD,FUT,SI,SI,True,F,0.001,5000.0,1.0,0.0,2,76
8215,204334,BUSJ1-BUSV1,2021-04-12 19:15:00,SPREAD,FUT,MA,BUS,True,F,0.010,20.0,100.0,0.0,2,76


In [16]:
tmp[list(filter(lambda x: '_p' in x, tmp.columns))].mean()

min_tr_p     18116.501719
max_tr_p     18254.071203
dbid_p       18264.967908
dask_p           0.000000
mbid_p       18271.507307
mask_p           0.000000
last_tr_p    18182.558596
dtype: float64

In [2]:
sql_conn = sqlite3.connect('/home/john/spartan_store/sim/sim.sqlite')
sec = load_security(360, '2021-04-05')
run = pd.read_sql('select * from run', sql_conn)

def load_trade_win():
    trade_win = pd.read_sql('select * from trade_win_short', sql_conn)
    trade_win = trade_win.loc[trade_win.start_eid != 0]
    trade_win = pd.merge(trade_win, run[['run_id', 'market_date']], on='run_id')
    # trade_win = pd.merge(trade_win, sec[['sid', 'symbol', 'asset', 'sec_group', 'sec_type']], on='sid')
    trade_win = pd.merge(trade_win, sec[['sid', 'symbol']], on='sid')
    trade_win['market_date'] = trade_win['market_date'].astype('datetime64[ns]')
    trade_win['start_t'] = trade_win['start_t'].astype('datetime64[ns]')
    trade_win['stop_t'] = trade_win['stop_t'].astype('datetime64[ns]')
    trade_win['dbid_t'] = trade_win['dbid_t'].astype('datetime64[ns]')
    trade_win['dask_t'] = trade_win['dask_t'].astype('datetime64[ns]')
    trade_win['mbid_t'] = trade_win['mbid_t'].astype('datetime64[ns]')
    trade_win['mask_t'] = trade_win['mask_t'].astype('datetime64[ns]')
    return trade_win

trade_win = load_trade_win()

In [5]:
tmp = trade_win.loc[(trade_win['market_date'] == '2021-06-09') & (trade_win['symbol'] == 'GCQ1')].copy()
# trade_win['symbol'].unique()
tmp

,run_id,sid,start_eid,start_t,stop_t,mean_tr_p,volume,min_tr_p,max_tr_p,dbid_p,dbid_eid,dbid_t,dask_p,dask_eid,dask_t,mbid_p,mbid_eid,mbid_t,mask_p,mask_eid,mask_t,last_tr_p,last_tr_eid,last_tr_t,fill_qbp,market_date,symbol
1630,6,152,2971,2021-06-09 22:00:00.000000000,2021-06-09 22:04:51.570487296,18911.8,190.0,18909.0,18915.0,18914.0,7368.0,2021-06-09 22:00:30.504657920,18910.0,10346.0,2021-06-09 22:03:16.930721280,18914.0,7368.0,2021-06-09 22:00:30.504657920,18910.0,10346.0,2021-06-09 22:03:16.930721280,18913.0,11428.0,1.623276e+18,"18909,12;18910,35;18911,36;18912,26;18913,57;1...",2021-06-09,GCQ1
1631,6,152,11988,2021-06-09 22:05:16.829425401,2021-06-09 22:10:05.959051520,18911.3,171.0,18908.0,18914.0,18913.0,13065.0,2021-06-09 22:06:00.215894784,18908.0,15365.0,2021-06-09 22:10:05.888247296,18913.0,13065.0,2021-06-09 22:06:00.215894784,18908.0,15365.0,2021-06-09 22:10:05.888247296,18908.0,15424.0,1.623277e+18,"18908,24;18909,19;18910,19;18911,17;18912,17;1...",2021-06-09,GCQ1
1632,6,152,15504,2021-06-09 22:10:25.045908221,2021-06-09 22:14:40.952114432,18908.6,75.0,18907.0,18910.0,18910.0,16540.0,2021-06-09 22:12:02.272574720,18908.0,16251.0,2021-06-09 22:11:53.671700480,18910.0,16540.0,2021-06-09 22:12:02.272574720,18908.0,16251.0,2021-06-09 22:11:53.671700480,18910.0,17681.0,1.623277e+18,"18907,9;18908,25;18909,25;18910,16",2021-06-09,GCQ1
1633,6,152,17849,2021-06-09 22:15:27.723967881,2021-06-09 22:20:00.206166272,18910.9,34.0,18910.0,18911.0,18911.0,19332.0,2021-06-09 22:20:00.205681152,18911.0,19019.0,2021-06-09 22:19:28.335731200,18911.0,19332.0,2021-06-09 22:20:00.205681152,18911.0,19019.0,2021-06-09 22:19:28.335731200,18911.0,19333.0,1.623277e+18,"18910,5;18911,29",2021-06-09,GCQ1
1634,6,152,19466,2021-06-09 22:20:37.753181427,2021-06-09 22:25:16.519255552,18908.6,48.0,18907.0,18911.0,18910.0,19944.0,2021-06-09 22:22:29.629749248,18908.0,20968.0,2021-06-09 22:25:10.670176768,18910.0,19944.0,2021-06-09 22:22:29.629749248,18908.0,20968.0,2021-06-09 22:25:10.670176768,18909.0,21092.0,1.623278e+18,"18907,16;18908,12;18909,5;18910,7;18911,8",2021-06-09,GCQ1
1635,6,152,21157,2021-06-09 22:25:42.300403379,2021-06-09 22:30:38.188677376,18909.7,46.0,18908.0,18911.0,18910.0,23413.0,2021-06-09 22:30:38.188677376,18909.0,21395.0,2021-06-09 22:26:20.305339648,18910.0,23413.0,2021-06-09 22:30:38.188677376,18909.0,21395.0,2021-06-09 22:26:20.305339648,18911.0,23413.0,1.623278e+18,"18908,4;18909,11;18910,25;18911,6",2021-06-09,GCQ1
1636,6,152,23539,2021-06-09 22:31:00.132922739,2021-06-09 22:35:48.043215104,18909.3,63.0,18908.0,18911.0,18910.0,25458.0,2021-06-09 22:35:48.043215104,18908.0,25024.0,2021-06-09 22:35:23.382011392,18910.0,25458.0,2021-06-09 22:35:48.043215104,18908.0,25024.0,2021-06-09 22:35:23.382011392,18910.0,25458.0,1.623278e+18,"18908,17;18909,14;18910,29;18911,3",2021-06-09,GCQ1
1637,6,152,25755,2021-06-09 22:36:42.766770027,2021-06-09 22:41:33.544661248,18910.5,27.0,18909.0,18913.0,18912.0,27333.0,2021-06-09 22:41:27.673534976,18910.0,26324.0,2021-06-09 22:38:59.285682432,18912.0,27333.0,2021-06-09 22:41:27.673534976,18910.0,26324.0,2021-06-09 22:38:59.285682432,18913.0,27371.0,1.623278e+18,"18909,4;18910,12;18911,6;18912,3;18913,2",2021-06-09,GCQ1
1638,6,152,27464,2021-06-09 22:41:43.832463309,2021-06-09 22:46:39.186141952,18912.6,43.0,18911.0,18913.0,18912.0,29336.0,2021-06-09 22:46:39.186141952,18912.0,28311.0,2021-06-09 22:44:02.145506304,18912.0,29336.0,2021-06-09 22:46:39.186141952,18912.0,28311.0,2021-06-09 22:44:02.145506304,18912.0,29336.0,1.623279e+18,"18911,1;18912,15;18913,27",2021-06-09,GCQ1
1639,6,152,29360,2021-06-09 22:46:44.398747541,2021-06-09 22:51:02.137788416,18913.5,28.0,18912.0,18915.0,18914.0,30638.0,2021-06-09 22:49:52.052957184,18913.0,30960.0,2021-06-09 22:51:02.137788416,18914.0,30638.0,2021-06-09 22:49:52.052957184,18913.0,30960.0,2021-06-09 22:51:02.137788416,18913.0,30960.0,1.623279e+18,"18912,3;18913,10;18914,14;18915,1",2021-06-09,GCQ1


In [166]:
gc_sgu = pd.read_feather(PCapLocationParams(360).feather_file_path(name='gc_sgu_bbo', date='2021-06-09'))

FileNotFoundError: [Errno 2] No such file or directory: '/home/john/spartan_store/pcaps/360/2021-06-09_360_24h_feather/gc_sgu_bbo.feather'

In [25]:
def get_date_picker(dates, fmt = None):
    if dates is None or len(dates) == 0:
        raise ValueError(f'Invalid parameter \'dates\': {dates} ')
    opts = [(date.strftime(fmt), date) for date in dates]
    selection_range_slider = ipw.SelectionRangeSlider(
        options=opts,
        index=(0, len(opts) - 1),
        description='Dates',
        orientation='horizontal',
        layout={'width': '500px'}
    )
    return selection_range_slider

def parse_fill_by_price(x):
    price_qtys = x['fill_qbp'].split(';')
    prices = []
    qtys = []
    for pq in price_qtys:
        p, q = pq.split(',')
        prices.append(float(p))
        qtys.append(int(q))
    start_eid = np.empty(len(prices))
    start_eid.fill(x['start_eid'])
    start_t = pd.Series(np.full(len(prices), x['start_t']), name='start_t', dtype='datetime64[ns]')
#     start_t = x['start_t'].repeat(len(prices))
    stop_t = pd.Series(np.full(len(prices), x['stop_t']), name='stop_t', dtype='datetime64[ns]')

    mean_tr_p = pd.Series(np.empty(len(prices)), name='mean_tr_p', dtype='float')
    mean_tr_p = x['mean_tr_p']

    return pd.DataFrame(data={'price': prices, 'qty': qtys, 'start_eid': start_eid, 'start_t': start_t, 'stop_t': stop_t, 'mean_tr_p': mean_tr_p})

def add_quote_prices(x, edge = 0.25):
    df = pd.DataFrame({'bid_quote': x['mean_tr_p'].shift() - edge, 'ask_quote': x['mean_tr_p'].shift() + edge })
#     x['bid_quote'] = df['bid_quote']
#     x['ask_quote'] = df['ask_quote']
    return df

def get_weighted_mean(df, value, weight, total_weight = None):
    if total_weight is None:
        total_weight = df[weight].sum()
#     if isinstance()
    v = (df[value] * (df[weight] / total_weight)).sum()
    return v

In [17]:
def count_fills(x):
    def impl(y):
        fill_by_p = parse_fill_by_price(y)
#         fill_by_p = add_quote_prices(fill_by_p)

#         print(y)
        def get_fill_avg(side, reverse_cmp = False):
            fills = fill_by_p.loc[fill_by_p['price'] >= fill_by_p[f'{side}_quote']] if reverse_cmp else\
                fill_by_p.loc[fill_by_p['price'] <= fill_by_p[f'{side}_quote']]
            fill_q = fills['qty'].sum()
            fill_p = (fills['price'] * fills['qty'] / fill_q).sum()

        fill_dict = {}
        bid_fills = fill_by_p.loc[fill_by_p['price'] <= y['bid_quote']]
        bid_fill_q = bid_fills['qty'].sum()
        bid_fill_tr_p = (bid_fills['price'] * bid_fills['qty'] / bid_fill_q).sum()
        fill_dict['bid_fills'] = bid_fills
        fill_dict['bid_fill_p'] = y['bid_quote']
        fill_dict['bid_fill_tr_p'] = bid_fill_tr_p
        fill_dict['bid_fill_q'] = bid_fill_q

        ask_fills = fill_by_p.loc[fill_by_p['price'] >= y['ask_quote']]
        ask_fill_q = ask_fills['qty'].sum()
        ask_fill_tr_p = (ask_fills['price'] * ask_fills['qty'] / ask_fill_q).sum()
        fill_dict['ask_fills'] = ask_fills
        fill_dict['ask_fill_p'] = y['ask_quote']
        fill_dict['ask_fill_tr_p'] = ask_fill_tr_p
        fill_dict['ask_fill_q'] = ask_fill_q

        sides = ['bid', 'ask']
        return pd.DataFrame(data=dict(
            [(f'{s}_fill_p', [fill_dict[f'{s}_fill_p']]) for s in sides] +
            [(f'{s}_fill_q', [fill_dict[f'{s}_fill_q']]) for s in sides]))

#     shift = x.shift(1)
#     shift = shift.rename(columns={c: f'{c}_shift' for c in shift.columns})
#     print(shift.info())
#     res = pd.concat(objs=[x, shift], axis=1)
#     print(res.info())
    res = x.apply(impl, axis=1)
    res = pd.concat(objs=res.values).reset_index(drop=True)
    return res

def get_pnl(fills):
    bid_q = fills['bid_fill_q'].sum()
    bid_p = get_weighted_mean(fills, 'bid_fill_p', 'bid_fill_q', total_weight = bid_q)
    ask_q = fills['ask_fill_q'].sum()
    ask_p = get_weighted_mean(fills, 'ask_fill_p', 'ask_fill_q', total_weight = ask_q)
    realized_q = min(bid_q, ask_q)
    long_q = bid_q - realized_q
    short_q = ask_q - realized_q
    return pd.DataFrame(data={
        'bid_p': [bid_p], 'bid_q': [bid_q],
        'ask_p': [ask_p], 'ask_q': [ask_q],
        'realized_pnl': [(ask_p - bid_p) * realized_q],
        'pos_p': [bid_p if long_q > short_q else ask_p],
        'pos_q': [long_q - short_q]
    })

def get_fill_by_price(x):
    fbp = x[['start_eid', 'start_t', 'stop_eid', 'stop_t', 'fill_qbp']].apply(parse_fill_by_price)
    return fbp

def compute_pnl(df, edge):
    qps = add_quote_prices(df, edge)
    df = pd.concat(objs=[df, qps])
    fills = count_fills(df)
    pnl = get_pnl(df)
    return pnl

In [13]:
gcq1 = trade_win.loc[(trade_win.market_date == '2021-06-09') & (trade_win.symbol == 'GCQ1')].copy()

In [15]:
qps = add_quote_prices(gcq1, 0.75)
gcq1 = pd.concat(objs=[gcq1, qps], axis=1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 272 entries, 1630 to 1901
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   run_id       272 non-null    int64         
 1   sid          272 non-null    int64         
 2   start_eid    272 non-null    int64         
 3   start_t      272 non-null    datetime64[ns]
 4   stop_t       272 non-null    datetime64[ns]
 5   mean_tr_p    272 non-null    float64       
 6   volume       272 non-null    float64       
 7   min_tr_p     272 non-null    float64       
 8   max_tr_p     272 non-null    float64       
 9   dbid_p       272 non-null    float64       
 10  dbid_eid     272 non-null    float64       
 11  dbid_t       272 non-null    datetime64[ns]
 12  dask_p       272 non-null    float64       
 13  dask_eid     272 non-null    float64       
 14  dask_t       272 non-null    datetime64[ns]
 15  mbid_p       272 non-null    float64       
 16  mbid

In [164]:
fills

,bid_fill_p,ask_fill_p,bid_fill_q,ask_fill_q
0,18935.15,18935.65,268,163
1,18930.15,18930.65,117,84
2,18926.65,18927.15,34,41
3,18922.75,18923.25,61,51
4,18923.45,18923.95,22,28
5,18923.85,18924.35,8,6
6,18927.05,18927.55,65,59
7,18926.45,18926.95,12,17
8,18921.65,18922.15,31,32
9,18922.15,18922.65,24,25


In [26]:
fills = count_fills(gcq1)

In [33]:
pd.DataFrame(data={'bid_fill_p': fills['bid_fill_p'], 'bid_fill_p_sh': fills['bid_fill_p'].shift()})

,bid_fill_p,bid_fill_p_sh
0,NaN,NaN
1,18911.05,NaN
2,18910.55,18911.05
3,18907.85,18910.55
4,18910.15,18907.85
5,18907.85,18910.15
6,18908.95,18907.85
7,18908.55,18908.95
8,18909.75,18908.55
9,18911.85,18909.75


In [165]:
fills[['bid_fill_q', 'ask_fill_q']].sum()

bid_fill_q    13348
ask_fill_q    12887
dtype: int64

In [142]:
fills

,bid_fill_p,ask_fill_p,bid_fill_q,ask_fill_q
0,18934.65,18936.15,238,163
1,18929.65,18931.15,79,84
2,18926.15,18927.65,34,41
3,18922.25,18923.75,61,51
4,18922.95,18924.45,18,28
5,18923.35,18924.85,8,6
6,18926.55,18928.05,48,59
7,18925.95,18927.45,1,17
8,18921.15,18922.65,31,32
9,18921.65,18923.15,12,25


In [88]:
pnl = get_pnl(fills)

KeyError: 'bid_fill_q'

In [ ]:
def get_theo_position(x):
    volume = x['volume']
    minp = x['min_tr_p']
    maxp = x['max_tr_p']
    mean = x['mean_tr_p']
    if min_p == mean or volume == 0:
        return (0, 0)
    

In [19]:
# imports
import bqplot

import ipywidgets as ipw
from ipywidgets import interactive
import pandas as pd
import numpy as np
from IPython.display import clear_output
import matplotlib
from matplotlib import pyplot as plt

from IPython.display import display

tmp = trade_win.copy()
tmp= tmp.loc[tmp.symbol.isin(['GCQ1'])]
print(len(tmp))

output = ipw.Output()
timespan_selector = get_date_picker(
    pd.date_range(datetime.datetime(2021, 6, 8, 17), datetime.datetime(2021, 6, 9, 15), freq='min'), fmt=' %H:%M ')

series_selector = ipw.SelectMultiple(
    # options=['bid_p_1', 'ask_p_1'],
    options=list(tmp.columns),
    value=['min_tr_p', 'max_tr_p'],
    rows=4,
    description='Variables')
ipw.FloatSlider()

display(series_selector)
display(timespan_selector)
display(output)

errors = []
echo = []
def multiplot(x):
    try:
        start = timespan_selector.value[0]
        stop = timespan_selector.value[1]
        data = tmp.loc[(tmp['start_t'] >= start) & (tmp['start_t'] <= stop)]
        cols = series_selector.value
        if cols is not None and len(cols) > 0:
            data = data[['start_t'] + list(cols)]
        output.clear_output(wait=True)
        with output:
            data.plot(kind='line', x='start_t', y=None, ylabel='price', figsize=(50,10), xlim=(start, stop))
            plt.show()
    except Exception as e:
        errors.append(e)


multiplot({'new': timespan_selector.value})
timespan_selector.observe(multiplot, names='value')
series_selector.observe(multiplot, names='value')

8305


SelectMultiple(description='Variables', index=(7, 8), options=('run_id', 'sid', 'start_eid', 'start_t', 'stop_…

SelectionRangeSlider(description='Dates', index=(0, 1320), layout=Layout(width='500px'), options=((' 17:00 ', …

Output()

In [12]:
# imports
import bqplot

import ipywidgets as ipw
from ipywidgets import interactive
import pandas as pd
import numpy as np
from IPython.display import clear_output
import matplotlib
from matplotlib import pyplot as plt

from IPython.display import display

tmp = trade_win.copy()
tmp= tmp.loc[tmp.symbol.isin(['GCQ1'])]
print(len(tmp))

output = ipw.Output()
timespan_selector = get_date_picker(
    pd.date_range(datetime.datetime(2021, 6, 9, 2), datetime.datetime(2021, 6, 9, 9), freq='min'), fmt=' %H:%M ')

series_selector = ipw.SelectMultiple(
    # options=['bid_p_1', 'ask_p_1'],
    options=list(tmp.columns),
    value=['min_tr_p', 'max_tr_p'],
    rows=2,
    description='Variables')
ipw.FloatSlider()

display(series_selector)
display(timespan_selector)
display(output)

errors = []
echo = []
def multiplot(x):
    try:
        start = timespan_selector.value[0]
        stop = timespan_selector.value[1]
        data = tmp.loc[(tmp['start_t'] >= start) & (tmp['start_t'] <= stop)]
        cols = series_selector.value
        if cols is not None and len(cols) > 0:
            data = data[['start_t'] + list(cols)]
        output.clear_output(wait=True)
        with output:
            data.plot(kind='line', x='start_t', y=None, ylabel='price', figsize=(50,10), xlim=(start, stop))
            plt.show()
    except Exception as e:
        errors.append(e)


multiplot({'new': timespan_selector.value})
timespan_selector.observe(multiplot, names='value')
series_selector.observe(multiplot, names='value')

2913


SelectMultiple(description='Variables', index=(5, 6), options=('run_id', 'sid', 'start_eid', 'start_t', 'stop_…

SelectionRangeSlider(description='Dates', index=(0, 420), layout=Layout(width='500px'), options=((' 02:00 ', T…

Output()